In [42]:
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, AdaLoraConfig, TaskType

df = pd.read_json('./dataset/input.json')
ds = Dataset.from_pandas(df)

In [43]:
tokenizer = AutoTokenizer.from_pretrained('./Qwen/Qwen2.5-Coder-0.5B-Instruct', use_fast=False, trust_remote_code=True)

# 数据处理函数
def process_func(example):
    MAX_LENGTH = 4096    
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer(f"<|im_start|>system\n你是一个专业的代码专家，熟悉AJX语言，熟悉高德地图前端开发，熟悉公共交通业务，能够根据高德地图前端开发需求，开发代码。<|im_end|>\n<|im_start|>user\n{example['instruction'] }<|im_end|>\n<|im_start|>assistant\n", add_special_tokens=False)  # add_special_tokens 不在开头加 special_tokens
    response = tokenizer(f"{example['output']}", add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]  
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]  
    if len(input_ids) > MAX_LENGTH: 
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }
tokenizer

Qwen2Tokenizer(name_or_path='./Qwen/Qwen2.5-Coder-0.5B-Instruct', vocab_size=151643, model_max_length=32768, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>', '<|object_ref_start|>', '<|object_ref_end|>', '<|box_start|>', '<|box_end|>', '<|quad_start|>', '<|quad_end|>', '<|vision_start|>', '<|vision_end|>', '<|vision_pad|>', '<|image_pad|>', '<|video_pad|>']}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151646: AddedToken("<|object_ref_start|>", rstrip=False, lstrip=False, single_word=Fa

In [ ]:
tokenized_id = ds.map(process_func, remove_columns=ds.column_names)

# 加载基础模型
import torch
model = AutoModelForCausalLM.from_pretrained(
    './Qwen/Qwen2.5-Coder-0.5B-Instruct/',
    device_map="auto",
    torch_dtype=torch.bfloat16
)

# 配置AdaLora适配器
peft_config = AdaLoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,  # 适配器秩
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    init_r=12,  # 初始秩
    target_r=8,  # 目标秩
    beta1=0.85,  # AdaLora特有的超参数
    beta2=0.85,
    tinit=200,  # 预热步数
    tfinal=1000,  # 最终步数
    deltaT=10,  # 更新频率
)

# 创建PeftModel
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
model.config.use_cache = False

# 训练参数配置
training_args = TrainingArguments(
    output_dir="./output/Qwen2.5-Coder-0.5B-Instruct-Adapter",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=12,
    learning_rate=1e-3,
    num_train_epochs=2,
    logging_steps=10,
    save_steps=10,
    gradient_checkpointing=True,
    # AdaLora相关的优化器设置
    optim="adamw_torch",
    warmup_ratio=0.1,
)

# 创建Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

Map:   0%|          | 0/508 [00:00<?, ? examples/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


trainable params: 6,600,672 || all params: 500,633,608 || trainable%: 1.3185


In [4]:
# 开始训练
trainer.train()

  0%|          | 0/84 [00:00<?, ?it/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/Users/liuchengzhuo/.pyenv/versions/3.10.14/lib/python3.10/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


{'loss': 15.7043, 'grad_norm': 0.515550434589386, 'learning_rate': 0.0009866666666666667, 'epoch': 0.24}


/Users/liuchengzhuo/.pyenv/versions/3.10.14/lib/python3.10/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


{'loss': 8.8418, 'grad_norm': 0.5652494430541992, 'learning_rate': 0.0008533333333333334, 'epoch': 0.47}


...

/Users/liuchengzhuo/.pyenv/versions/3.10.14/lib/python3.10/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


{'loss': 2.8914, 'grad_norm': 0.6293167471885681, 'learning_rate': 5.333333333333334e-05, 'epoch': 1.89}


/Users/liuchengzhuo/.pyenv/versions/3.10.14/lib/python3.10/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


{'train_runtime': 18814.324, 'train_samples_per_second': 0.054, 'train_steps_per_second': 0.004, 'train_loss': 5.819442703610375, 'epoch': 1.98}


TrainOutput(global_step=84, training_loss=5.819442703610375, metrics={'train_runtime': 18814.324, 'train_samples_per_second': 0.054, 'train_steps_per_second': 0.004, 'total_flos': 1245372634329840.0, 'train_loss': 5.819442703610375, 'epoch': 1.984251968503937})

## 测试模型效果

In [61]:
# 推理部分
def generate_response(is_finetune, prompt):
    from transformers import AutoModelForCausalLM, AutoTokenizer
    from peft import PeftModel
    import torch

    mode_path = 'Qwen/Qwen2.5-Coder-0.5B-Instruct/'
    adapter_path = 'output/Qwen2.5-Coder-0.5B-Instruct-Adapter/checkpoint-84/'

    # 加载tokenizer
    tokenizer = AutoTokenizer.from_pretrained(mode_path, trust_remote_code=True)

    # 加载基础模型
    model = AutoModelForCausalLM.from_pretrained(
        mode_path, 
        device_map="auto",
        torch_dtype=torch.bfloat16,
        trust_remote_code=True
    ).eval()
    if is_finetune:
        # 加载adapter权重
        model = PeftModel.from_pretrained(model, adapter_path)

    inputs = tokenizer.apply_chat_template(
        [
        # {"role":"system","content":"你是一个专业的代码专家，熟悉AJX语言，熟悉高德地图前端开发，熟悉公共交通业务，能够根据高德地图前端开发需求，开发代码。"},
            {"role": "user", "content": prompt}
            ],
        add_generation_prompt=True,
        tokenize=True,
        return_tensors="pt",
        return_dict=True
    ).to('mps')

    gen_config = {"max_length": 4096, "do_sample": True, "top_k": 1}

    with torch.no_grad():
        outputs = model.generate(**inputs, **gen_config)
        outputs = outputs[:, inputs['input_ids'].shape[1]:]
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [63]:
# 测试生成
test_prompt = "在高德地图前端公共交通业务中，如何计算和记录用户在页面上的停留时间，并支持暂停/恢复计时功能？"
response1 = generate_response(False, test_prompt)
print("原模型回答：\n",response1)
response2 = generate_response(True, test_prompt)
print("adalora微调模型回答：\n",response2)

原模型回答：
 在高德地图前端公共交通业务中，计算和记录用户在页面上的停留时间并支持暂停/恢复计时功能可以通过以下步骤实现：

1. **创建一个计时器**：使用JavaScript创建一个计时器来记录用户停留的时间。

2. **监听页面滚动事件**：当用户滚动页面时，更新计时器的值。

3. **暂停计时**：当用户暂停滚动时，停止计时器。

4. **恢复计时**：当用户恢复滚动时，重新开始计时器。

5. **显示计时结果**：在页面上显示用户停留的时间。

以下是一个简单的示例代码，展示了如何实现这些功能：

```javascript
// 创建一个计时器
var timer = null;

// 计算停留时间
function calculateStayTime() {
    var startTime = new Date();
    var endTime = new Date();
    var timeDifference = endTime - startTime;
    return timeDifference / 1000; // 将秒转换为毫秒
}

// 监听页面滚动事件
window.addEventListener('scroll', function() {
    if (timer) {
        clearInterval(timer);
    }
    timer = setInterval(calculateStayTime, 1000); // 每秒更新一次停留时间
});

// 暂停计时
function pauseTimer() {
    if (timer) {
        clearInterval(timer);
        timer = null;
    }
}

// 恢复计时
function resumeTimer() {
    if (timer) {
        clearInterval(timer);
        timer = setInterval(calculateStayTime, 1000); // 每秒更新一次停留时间
    }
}

// 显示计时结果
function displayStayTime() {
    var s